### Matrix Product States

In [1]:
import numpy as np
import scipy as sp

from ED_Functions import vec_norm,Hamiltonian_TFIM_open,groundState_Krylov

import time
import matplotlib.pyplot as plt

In [7]:
# TESTING
#J,g,N = 1,0.5,8
#
#L = 50
#
#H,basis = Hamiltonian_TFIM_open(J,g,N,return_basis=True)
#
#gs,psi = groundState_Krylov(J,g,N,L)
#gs,psi;
#
#
##A = np.empty((N,bondDim,bondDim))
##A.shape
#A = []
#bondDim = 1000
#
#alpha,beta = 2,2**(N-1)
#
#for D in range(N-1):
#    R = psi.reshape((alpha,beta))
#    
#    U,S,Vh = sp.linalg.svd(R,full_matrices=False)
#
#    #print(D,R.shape)
#    #print(U.shape,S.shape,Vh.shape)
#
#    if(len(S)>bondDim):
#        U = U[:,:bondDim]
#        S = S[:bondDim]
#        Vh = Vh[:bondDim,:]
#    
#    #print(U.shape,S.shape,Vh.shape)
#    #print("-------------------")
#
#    k = min(*U.shape,bondDim)
#    A.append(U[:k,:k])
#    alpha *= 2
#    beta //= 2
#
#    R = S@Vh
#
#print("MPS Shapes: ", [a.shape for a in A])
#print("Max Bond Dimension w/o truncation: ",2**(N/2))

In [187]:
def get_mps(psi,N,bondDim = 1e10):
    
    A = []
    alpha = 2
    R = psi.copy()
    for i in range(N):

        #print(i,R.shape)
        
        R = R.reshape((alpha,R.size//alpha))
        
        U,S,Vh = sp.linalg.svd(R,full_matrices=False)
    
        #print(R.shape)
        #print(U.shape,S.shape,Vh.shape)
    
        if(len(S)>bondDim):
            U = U[:,:bondDim]
            S = S[:bondDim]
            Vh = Vh[:bondDim,:]
            
        #print(U.shape,S.shape,Vh.shape)
        #print("-------------------")
    
        #k = min(*U.shape,bondDim)
        #A.append(U[:k,:k])
        A.append(U)

        alpha = 2*len(S)
    
        R = np.diag(S)@Vh
        
    return A

In [188]:
J,g,N = 1,0.5,8
L = 50

H,basis = Hamiltonian_TFIM_open(J,g,N,return_basis=True)

gs,psi = groundState_Krylov(J,g,N,L)

psi_mps = get_mps(psi,N,100)
[x.shape for x in psi_mps],len(psi_mps),[np.diag((x.T @ x)) for x in psi_mps]

([(2, 2), (4, 4), (8, 8), (16, 16), (32, 8), (16, 4), (8, 2), (4, 1)],
 8,
 [array([1., 1.]),
  array([1., 1., 1., 1.]),
  array([1., 1., 1., 1., 1., 1., 1., 1.]),
  array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
  array([1., 1., 1., 1., 1., 1., 1., 1.]),
  array([1., 1., 1., 1.]),
  array([1., 1.]),
  array([1.])])

In [190]:
rng = np.random.default_rng(seed = 123)

N = 10
psi_random = rng.random(2**10)
psi_random = psi_random/vec_norm(psi_random)
psi_random_mps = get_mps(psi_random,N,20)
[x.shape for x in psi_random_mps]

[(2, 2),
 (4, 4),
 (8, 8),
 (16, 16),
 (32, 20),
 (40, 16),
 (32, 8),
 (16, 4),
 (8, 2),
 (4, 1)]

In [191]:
[np.diag((x.T @ x)) for x in psi_random_mps]

[array([1., 1.]),
 array([1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1., 1., 1., 1., 1.]),
 array([1., 1., 1., 1.]),
 array([1., 1.]),
 array([1.])]